# Tarea 2: utilizar SageMaker Clarify
	
En este laboratorio, utilice Amazon SageMaker Clarify para detectar sesgos en los datos previos al entrenamiento y en los modelos posteriores al entrenamiento, así como para acceder a los informes de explicabilidad.


## Tarea 2.1: configuración del entorno

Instale los paquetes y las dependencias.

In [ ]:
#install-dependencies

import boto3
import io
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import sagemaker
import sagemaker_datawrangler
import sys
import time
import zipfile

from IPython.display import display
from IPython.display import Image
from sagemaker import clarify
from sagemaker import Session
from sagemaker.inputs import TrainingInput
from sagemaker.s3 import S3Uploader
from sklearn.model_selection import train_test_split
from time import gmtime, strftime

s3_client = boto3.client("s3")
session = Session()
bucket = session.default_bucket()
prefix = 'sagemaker/lab_8'
role = sagemaker.get_execution_role()

A continuación, importe, divida y cargue el conjunto de datos.

In [ ]:
#prepare-dataset

lab_test_data = pd.read_csv('adult_data_processed.csv')

# Split the dataset
train_data, validation_data, test_data = np.split(
    lab_test_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(lab_test_data)), int(0.9 * len(lab_test_data))],
)

train_data.to_csv('train_data.csv', index=False, header=False)
validation_data.to_csv('validation_data.csv', index=False, header=False)
test_data.to_csv('test_data.csv', index=False, header=False)

# Upload the Dataset to S3
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

sagemaker_session = sagemaker.Session()

train_path = S3Uploader.upload('train_data.csv', 's3://{}/{}'.format(bucket, prefix))
validation_path = S3Uploader.upload('validation_data.csv', 's3://{}/{}'.format(bucket, prefix))
test_path = S3Uploader.upload('test_data.csv', 's3://{}/{}'.format(bucket, prefix))

train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')
test_input = TrainingInput(validation_path, content_type='text/csv')

data_inputs = {
    'train': train_input,
    'validation': validation_input
}

Ahora, entrene el modelo de XGBoost. Utilice este modelo entrenado para el objeto ModelConfig de SageMaker Clarify. El tiempo estimado para completar este entrenamiento es de aproximadamente 3 a 4 minutos.

In [ ]:
#train-model

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)
# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    sagemaker_session=sagemaker_session
)
# Set the hyperparameters
xgb.set_hyperparameters(
    max_depth=5, 
    eta=0.2, 
    gamma=4, 
    min_child_weight=6,
    subsample=0.8, 
    verbosity=1, 
    objective='binary:logistic', 
    num_round=800
)

# Train the model
xgb.fit(
    inputs = data_inputs
) 

## Tarea 2.2: configurar el trabajo de SageMaker Clarify y acceder al informe de sesgos

Para usar [SageMaker Clarify](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-configure-processing-jobs.html), primero cree un modelo a partir de su trabajo de entrenamiento. Luego, establezca las configuraciones necesarias y ejecute el trabajo de SageMaker Clarify en su modelo entrenado. En esta tarea, complete lo siguiente:

- Cree el modelo.
- Habilite SageMaker Clarify.
- Ejecute el informe de sesgos.
- Acceda al informe.

### Tarea 2.2.1: crear el modelo

Cree un modelo a partir del trabajo de entrenamiento para usar SageMaker Clarify.

In [ ]:
#create-clarify-model

model_name = "lab-8-clarify-model"
model = xgb.create_model(name=model_name)
container_def = model.prepare_container_def()
session.create_model(model_name, role, container_def)

### Tarea 2.2.2: habilitar SageMaker Clarify

Ahora, para comenzar la configuración, habilite SageMaker Clarify.

In [ ]:
#enable-clarify

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge", 
    sagemaker_session=session
)

Con SageMaker Clarify, puede detectar posibles sesgos previos al entrenamiento en sus datos y posteriores al entrenamiento en sus modelos a través de diversas métricas. Consulte [métricas](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-configure-analysis.html) para obtener más información sobre la configuración del análisis de Clarify.

Configure el objeto **DataConfig** para comunicar información de E/S de los datos a SageMaker Clarify. Aquí, especifique la ubicación del conjunto de datos de entrada, la ubicación para la salida, la columna de ingresos (**etiqueta**), los nombres de encabezados y el tipo de conjunto de datos.

In [ ]:
#define-data-config

bias_report_output_path = "s3://{}/{}/clarify-bias".format(bucket, prefix)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=train_path,
    s3_output_path=bias_report_output_path,
    label="income",
    headers=train_data.columns.to_list(),
    dataset_type="text/csv",
)

Configure el objeto **ModelConfig** para comunicar información sobre su modelo entrenado. Establezca el nombre del modelo y un punto de enlace exclusivo temporal para evitar enviar tráfico adicional a su modelo de producción (**instance_type** e **instance_count**). Además, configure **accept_type** para indicar el formato de la carga de respuesta del punto de enlace y **content_type** para indicar el formato de la carga de la solicitud al punto del enlace.

In [ ]:
#define-model-config

model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    accept_type="text/csv",
    content_type="text/csv",
)

Configure el término **probability_threshold** del objeto **ModelPredictedLabelConfig** para convertir la probabilidad de las muestras en etiquetas binarias para el análisis de sesgos. La predicción que esté por encima del umbral se interpretará como valor de etiqueta **1** y la que sea igual al umbral o esté por debajo de este se interpretará como valor de etiqueta **0**.

In [ ]:
#define-model-predicted-label-config

predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

Configure el objeto **BiasConfig** para indicar las columnas sensibles (**facets**), las posibles características sensibles (**facet_values_or_threshold**) y los resultados favorables (**label_values_or_threshold**).

Puede señalar datos categóricos y continuos para **facet_values_or_threshold** y **label_values_or_threshold**. Las funciones **sex** (sexo) y **age** (edad) son categóricas.

El objetivo de este modelo es determinar si una persona gana 50 000 USD o más, donde un resultado positivo es favorable. Use **BiasConfig** para proporcionar información sobre las columnas que contienen las facetas con el grupo sensible de **Sex**, las funciones sensibles que podrían estar usando **facet_values_or_threshold** y los resultados deseados que están usando **label_values_or_threshold**. Agrupe por **Age** para ver si existen diferencias según la edad de las personas.

In [ ]:
#define-bias-config

bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1], facet_name="sex", facet_values_or_threshold=[0], group_name="age"
)

### Tarea 2.2.3: ejecutar el informe de sesgos

Cree el informe de sesgos usando sus configuraciones para análisis previos y posteriores al entrenamiento. El tiempo estimado de este paso es de 10 a 20 minutos. Mientras se genera el informe de sesgos, puede descubrir cómo SageMaker Clarify procesa [medidas de equidad para machine learning en Finanzas](./Fairness.Measures.for.Machine.Learning.in.Finance.pdf).

In [ ]:
#run-bias-report

clarify_processor.run_bias(
    data_config=bias_data_config,
    bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    pre_training_methods="all",
    post_training_methods="all",
)

### Tarea 2.2.4: acceder al informe de sesgos

En SageMaker Studio, puede ver los resultados debajo de la pestaña Experiments (Experimentos).

El próximo paso abre una nueva pestaña en SageMaker Studio. Para seguir esas instrucciones, utilice las siguientes opciones:
- **Opción 1**: ver las pestañas una al lado de la otra. Para crear una vista de pantalla dividida de la ventana principal de SageMaker Studio, arrastre la pestaña **lab_8.ipynb** hacia el lado o seleccione (con el botón derecho del mouse) la pestaña **lab_8.ipynb** y elija **New View for Notebook** (Nueva vista para el cuaderno). Ahora puede tener las instrucciones visibles a medida que explora el informe de sesgos.
- **Opción 2**: alternar entre las pestañas de SageMaker Studio para seguir estas instrucciones. Cuando termine de explorar el informe de sesgos, seleccione la pestaña **lab_8.ipynb** para volver al cuaderno.

1. En SageMaker Studio, seleccione el ícono de **SageMaker Home** (Inicio de SageMaker).

2. Ejecute **Experiments** (Experimentos).

Verá la pestaña **Experiments** (Experimentos) en SageMaker Studio.

3. En la pestaña **Experiments** (Experimentos), seleccione **Unassigned runs** (Ejecuciones no asignadas) a la izquierda.

4. En la lista **Unassigned runs**, seleccione el nombre del trabajo de entrenamiento que contiene **clarify-bias-** en el título. 

5. En la pestaña **Experiments** (Experimentos), seleccione **Bias Reports** (Informes de sesgos) a la izquierda.

El informe de sesgos de las métricas está disponible cuando SageMaker Clarify termina de ejecutar el trabajo de sesgos.

6. Espere a que SageMaker Clarify termine de ejecutar el trabajo de sesgos.

Cada métrica de sesgo tiene explicaciones detalladas con ejemplos que puede explorar si selecciona cada valor. 

7. Examine las explicaciones detalladas de **Class Imbalance** (Desequilibrios de clase) y **Disparate (Adverse) Impact (DI)** (Impacto dispar [DI] [adverso]) si elige las flechas junto a las métricas de sesgo y expande el campo. 

Cuando termine de explorar las métricas de sesgo, puede continuar con la siguiente tarea.

## Tarea 2.3: acceder a informes sobre la explicabilidad

Además de usar un informe de sesgos, puede usar SageMaker Clarify para analizar las explicaciones locales de las predicciones individuales. Cree un informe para revisar los resultados de explicabilidad de las predicciones producidas por SageMaker Clarify y analizar las métricas clave del informe. En esta tarea, complete lo siguiente:

1. Defina una configuración de valores de Shapley (SHAP).
2. Ejecute el informe de explicabilidad.
3. Acceda al informe de explicabilidad.

### Tarea 2.3.1: definir una configuración de SHAP

Puede ejecutar un informe de explicabilidad para obtener una explicación de por qué un modelo realizó una predicción específica. SageMaker Clarify utiliza SHAP para generar un informe sobre el razonamiento que el modelo hizo para tomar sus decisiones. Consulte [Valores de referencia de SHAP para la explicabilidad](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-feature-attribute-shap-baselines.html) para obtener más información sobre los valores de referencia de SHAP.

Las métricas de SHAP que están configuradas son las siguientes:
- **baseline**: una lista de filas o un URI de objeto de Amazon Simple Storage Service (Amazon S3), que se usará como el conjunto de datos de valores de referencia en el algoritmo de Kernel SHAP.
- **num_samples**: cantidad de muestras que se usarán en el algoritmo de Kernel SHAP. Este número determina el tamaño del conjunto de datos sintético generado para procesar los valores de SHAP.
- **agg_method: mean_abs**: media de los valores de SHAP absolutos para todas las instancias.
- **save_local_shap_values**: valor booleano para indicar si los valores de SHAP locales se deben guardar en la ubicación de salida.

Consulte [métricas de SHAP](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-configure-analysis.html) para obtener más información sobre las métricas usadas.

In [ ]:
#configure-shap

testing_data, clarify_data = train_test_split(test_data, test_size =0.005)
clarify_data = clarify_data.drop(columns=["income"])
clarify_data.to_csv('clarify_data.csv', index=False, header=False)
clarify_path = S3Uploader.upload('clarify_data.csv', 's3://{}/{}'.format(bucket, prefix))

shap_config = clarify.SHAPConfig(
    baseline=clarify_path,
    num_samples=15,
    agg_method="mean_abs",
    save_local_shap_values=True,
)

explainability_output_path = "s3://{}/{}/clarify-explainability".format(bucket, prefix)
explainability_data_config = clarify.DataConfig(
    s3_data_input_path=clarify_path,
    s3_output_path=explainability_output_path,
    headers=clarify_data.columns.to_list(),
    dataset_type="text/csv",
)

### Tarea 2.3.2: ejecutar el informe de explicabilidad

Use sus configuraciones para crear el informe de explicabilidad. El tiempo estimado de este paso es de 10 a 15 minutos. Mientras se genera el informe de explicabilidad, puede seguir con el proceso del estado del trabajo en SageMaker Studio si continúa con la siguiente tarea.

Consulte [Documento técnico sobre imparcialidad y explicabilidad de la IA de Amazon](./Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf) para obtener información sobre el proceso de explicabilidad de SageMaker Clarify.

In [ ]:
#run-explainability-report

clarify_processor.run_explainability(
    data_config=explainability_data_config,
    model_config=model_config,
    explainability_config=shap_config,
)

### Tarea 2.3.3: acceder al informe de explicabilidad

Al igual que con el informe de sesgos, puede revisar el informe de explicabilidad en Studio como parte de los experimentos.

1. Seleccione la pestaña **Experiments** (Experimentos).

2. En la pestaña **Experiments** (Experimentos), seleccione **Unassigned runs** (Ejecuciones no asignadas) a la izquierda.

3. En la lista **Unassigned runs** (Ejecuciones no asignadas), seleccione el nombre del trabajo de entrenamiento que contiene **clarify-bias-** en el título. 

4. En la pestaña **Experiments** (Experimentos), seleccione **Explainability** (Explicabilidad) a la izquierda.

5. Cuando haya finalizado el trabajo de explicabilidad, seleccione <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">View sample notebook</span> (Ver cuaderno de muestra) para ver el gráfico de **importancia de las funciones**.

SageMaker Studio abre el cuaderno **Imparcialidad y explicabilidad con SageMaker Clarify** en una pestaña.

¿Qué funciones tienen mayor y mejor importancia? ¿Hay algunos resultados que lo sorprendan?

### Conclusión

¡Felicitaciones! Usó SageMaker Clarify para crear informes de sesgos y explicabilidad que puede usar para desarrollar modelos más fiables y conformes. En el próximo laboratorio, implemente sus modelos y ejecute la inferencia. Seguirá trabajando con el modelo en el próximo laboratorio.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno.
- Regrese a la sesión de laboratorio y continúe con la **Conclusión**.